In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS retail.gold;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail.gold.product_daily (
  ProductId STRING,
  event_date DATE,
  review_count BIGINT,
  avg_rating DOUBLE,
  avg_sentiment DOUBLE,
  pct_negative DOUBLE,
  last_event_time TIMESTAMP
)
USING DELTA;

In [0]:
%sql
MERGE INTO retail.gold.product_daily t
USING (
  SELECT
    ProductId,
    event_date,
    COUNT(*)                                    AS review_count,
    AVG(rating)                                 AS avg_rating,
    AVG(sentiment_score)                        AS avg_sentiment,
    AVG(CASE WHEN sentiment_label='negative' THEN 1.0 ELSE 0.0 END) AS pct_negative,
    MAX(event_time)                             AS last_event_time
  FROM retail.clean.silver_reviews
  GROUP BY ProductId, event_date
) a
ON  t.ProductId = a.ProductId
AND t.event_date = a.event_date
WHEN MATCHED THEN UPDATE SET
  t.review_count    = a.review_count,
  t.avg_rating      = a.avg_rating,
  t.avg_sentiment   = a.avg_sentiment,
  t.pct_negative    = a.pct_negative,
  t.last_event_time = a.last_event_time
WHEN NOT MATCHED THEN INSERT *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
207778,0,0,207778


In [0]:
CHK_GOLD   = "abfss://retail-input-data@streaminputsa01.dfs.core.windows.net/_chk/silver_to_gold/product_daily"

In [0]:
#Turn on the incremental updater (stream)
q_gold = (
    silver_changes.writeStream
      .option("checkpointLocation", CHK_GOLD)
      .foreachBatch(upsert_product_daily)
      .start()
)

In [0]:
%sql
--Top 5 products by 7-day review volume (with weighted quality)

CREATE OR REPLACE VIEW retail.bi.v_top5_products_7d_by_reviews AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily)
SELECT
  p.ProductId,
  SUM(p.review_count) AS reviews_7d,
  ROUND(SUM(p.avg_rating   * p.review_count) / SUM(p.review_count), 2) AS w_avg_rating_7d,
  ROUND(SUM(p.avg_sentiment* p.review_count) / SUM(p.review_count), 3) AS w_avg_sentiment_7d,
  ROUND(SUM(p.pct_negative * p.review_count) / SUM(p.review_count), 3) AS w_neg_rate_7d
FROM retail.gold.product_daily p, mx
WHERE p.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
GROUP BY p.ProductId
ORDER BY reviews_7d DESC
LIMIT 5;


In [0]:
%sql
--Top 5 products by highest 7-day negative rate (min 30 reviews)

CREATE OR REPLACE VIEW retail.gold.v_top5_products_7d_by_neg_rate AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily),
agg AS (
  SELECT
    p.ProductId,
    SUM(p.review_count) AS reviews_7d,
    SUM(p.pct_negative * p.review_count) / NULLIF(SUM(p.review_count),0) AS w_neg_rate_7d
  FROM retail.gold.product_daily p, mx
  WHERE p.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
  GROUP BY p.ProductId
)
SELECT ProductId, reviews_7d,
       ROUND(w_neg_rate_7d, 3) AS w_neg_rate_7d
FROM agg
WHERE reviews_7d >= 30
ORDER BY w_neg_rate_7d DESC
LIMIT 5;



In [0]:
%sql
--Top 5 products with week-over-week spike in negative rate

CREATE OR REPLACE VIEW retail.gold.v_top5_products_wow_neg_spike AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily),
win1 AS (  -- last 7 days
  SELECT p.ProductId,
         SUM(p.review_count) AS rev1,
         SUM(p.pct_negative * p.review_count) / NULLIF(SUM(p.review_count),0) AS neg1
  FROM retail.gold.product_daily p, mx
  WHERE p.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
  GROUP BY p.ProductId
),
win0 AS (  -- prior 7 days
  SELECT p.ProductId,
         SUM(p.review_count) AS rev0,
         SUM(p.pct_negative * p.review_count) / NULLIF(SUM(p.review_count),0) AS neg0
  FROM retail.gold.product_daily p, mx
  WHERE p.event_date BETWEEN date_sub(mx.max_d, 13) AND date_sub(mx.max_d, 7)
  GROUP BY p.ProductId
)
SELECT
  w1.ProductId,
  w1.rev1 AS reviews_7d,
  ROUND(w1.neg1, 3) AS neg_rate_curr_7d,
  ROUND(w0.neg0, 3) AS neg_rate_prev_7d,
  ROUND(w1.neg1 - w0.neg0, 3) AS neg_rate_wow_delta
FROM win1 w1
JOIN win0 w0 USING (ProductId)
WHERE w1.rev1 >= 30 AND w0.rev0 >= 30
ORDER BY neg_rate_wow_delta DESC
LIMIT 5;


In [0]:
%sql

--Bottom 5 products by 7-day sentiment (min 30 reviews)

CREATE OR REPLACE VIEW retail.gold.v_bottom5_products_7d_by_sentiment AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily),
agg AS (
  SELECT
    p.ProductId,
    SUM(p.review_count) AS reviews_7d,
    SUM(p.avg_sentiment * p.review_count) / NULLIF(SUM(p.review_count),0) AS w_sent_7d
  FROM retail.gold.product_daily p, mx
  WHERE p.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
  GROUP BY p.ProductId
)
SELECT ProductId, reviews_7d,
       ROUND(w_sent_7d, 3) AS w_avg_sentiment_7d
FROM agg
WHERE reviews_7d >= 30
ORDER BY w_avg_sentiment_7d ASC
LIMIT 5;


In [0]:
%sql
--Latest-day negative leaderboard

CREATE OR REPLACE VIEW retail.gold.v_top5_products_latestday_by_neg_rate AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily)
SELECT
  p.ProductId,
  p.review_count AS reviews,
  ROUND(p.pct_negative, 3) AS neg_rate
FROM retail.gold.product_daily p, mx
WHERE p.event_date = mx.max_d
  AND p.review_count >= 10
ORDER BY neg_rate DESC, reviews DESC
LIMIT 5;


In [0]:
%sql
--Top 5 positive reviews on the latest day

CREATE OR REPLACE VIEW retail.gold.v_top5_positive_reviews_latestday AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily)
SELECT
  s.ProductId        AS product_id,
  s.event_time,
  s.rating,
  s.sentiment_score,
  s.Summary          AS summary,
  s.Text             AS review_text
FROM retail.clean.silver_reviews s, mx
WHERE s.sentiment_label = 'positive'
  AND s.event_date = mx.max_d
ORDER BY s.sentiment_score DESC, s.rating DESC, s.event_time DESC
LIMIT 5;

In [0]:
%sql
--Top 5 positive reviews in the last 7 days

CREATE OR REPLACE VIEW retail.gold.v_top5_positive_reviews_7d AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily)
SELECT
  s.ProductId        AS product_id,
  s.event_time,
  s.rating,
  s.sentiment_score,
  s.Summary          AS summary,
  s.Text             AS review_text
FROM retail.clean.silver_reviews s, mx
WHERE s.sentiment_label = 'positive'
  AND s.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
ORDER BY s.sentiment_score DESC, s.rating DESC, s.event_time DESC
LIMIT 5;


In [0]:
%sql
--Top 5 products by positive rate over 7 days (min 30 reviews)

CREATE OR REPLACE VIEW retail.gold.v_top5_products_7d_by_positive_rate AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily),
agg AS (
  SELECT
    s.ProductId,
    COUNT(*) AS reviews_7d,
    SUM(CASE WHEN s.sentiment_label='positive' THEN 1 ELSE 0 END) AS pos_7d
  FROM retail.clean.silver_reviews s, mx
  WHERE s.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
  GROUP BY s.ProductId
)
SELECT
  ProductId,
  reviews_7d,
  pos_7d,
  ROUND(pos_7d / CAST(reviews_7d AS DOUBLE), 3) AS pos_rate_7d
FROM agg
WHERE reviews_7d >= 30
ORDER BY pos_rate_7d DESC, reviews_7d DESC
LIMIT 5;


In [0]:
%sql
--Top 5 products by positive rate on the latest day (min 10 reviews)

CREATE OR REPLACE VIEW retail.gold.v_top5_products_latestday_by_positive_rate AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily),
agg AS (
  SELECT
    s.ProductId,
    COUNT(*) AS reviews_today,
    SUM(CASE WHEN s.sentiment_label='positive' THEN 1 ELSE 0 END) AS pos_today
  FROM retail.clean.silver_reviews s, mx
  WHERE s.event_date = mx.max_d
  GROUP BY s.ProductId
)
SELECT
  ProductId,
  reviews_today,
  pos_today,
  ROUND(pos_today / CAST(reviews_today AS DOUBLE), 3) AS pos_rate_today
FROM agg
WHERE reviews_today >= 10
ORDER BY pos_rate_today DESC, reviews_today DESC
LIMIT 5;


In [0]:
%sql
--Top 5 products by positive volume (count of positive reviews) over 7 days

CREATE OR REPLACE VIEW retail.gold.v_top5_products_7d_by_positive_volume AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily)
SELECT
  s.ProductId,
  SUM(CASE WHEN s.sentiment_label='positive' THEN 1 ELSE 0 END) AS pos_7d,
  COUNT(*) AS reviews_7d,
  ROUND(
    SUM(CASE WHEN s.sentiment_label='positive' THEN 1 ELSE 0 END)
    / CAST(COUNT(*) AS DOUBLE), 3
  ) AS pos_rate_7d
FROM retail.clean.silver_reviews s, mx
WHERE s.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
GROUP BY s.ProductId
ORDER BY pos_7d DESC, reviews_7d DESC
LIMIT 5;


In [0]:
%sql
--For the top 5 products (by 7-day volume), show top 3 positive reviews each (max 15 rows)

CREATE OR REPLACE VIEW retail.gold.v_top_products_7d_with_top3_positive_reviews AS
WITH mx AS (SELECT MAX(event_date) AS max_d FROM retail.gold.product_daily),
top_prod AS (  -- pick 5 busiest products in last 7 days
  SELECT s.ProductId
  FROM retail.clean.silver_reviews s, mx
  WHERE s.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
  GROUP BY s.ProductId
  ORDER BY COUNT(*) DESC
  LIMIT 5
),
ranked AS (
  SELECT
    s.ProductId AS product_id,
    s.event_time,
    s.rating,
    s.sentiment_score,
    s.Summary    AS summary,
    s.Text       AS review_text,
    ROW_NUMBER() OVER (
      PARTITION BY s.ProductId
      ORDER BY s.sentiment_score DESC, s.rating DESC, s.event_time DESC
    ) AS rn
  FROM retail.clean.silver_reviews s
  JOIN top_prod t ON s.ProductId = t.ProductId
  JOIN mx
  WHERE s.sentiment_label = 'positive'
    AND s.event_date BETWEEN date_sub(mx.max_d, 6) AND mx.max_d
)
SELECT product_id, event_time, rating, sentiment_score, summary, review_text
FROM ranked
WHERE rn <= 3
ORDER BY product_id, rn;
